In [ ]:
!pip install -U adapter-transformers
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset.num_rows

In [ ]:
dataset['train'][0]

In [ ]:
dataset

In [ ]:
from transformers import AutoConfig, AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def encode_batch(batch):
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

dataset = dataset.map(encode_batch, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
model


In [ ]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction, Trainer
from transformers.adapters import PrefixTuningConfig
from transformers import  BertModel, AutoModelForSequenceClassification




def compute_accuracy(p: EvalPrediction):
      preds = np.argmax(p.predictions, axis=1)
      return {"acc": (preds == p.label_ids).mean()}

def training(type_model):
  config = AutoConfig.from_pretrained(
      "bert-base-uncased",
      num_labels=2,
  )
  model = AutoModelForSequenceClassification.from_pretrained(
      "bert-base-uncased",
      config=config,
  )
  if(type_model==1):
    config = PrefixTuningConfig(flat=False, prefix_length=30)
    model.add_adapter("prefix_tuning", config=config)
    model.train_adapter("prefix_tuning")

    training_args = TrainingArguments(
        learning_rate=1e-4,
        num_train_epochs=6,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        logging_steps=200,
        output_dir="./training_output",
        overwrite_output_dir=True,
        # The next line is important to ensure the dataset labels are properly passed to the model
        remove_unused_columns=False,
    )

    

    trainer = AdapterTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        compute_metrics=compute_accuracy,
    )

  else:
    training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,)



    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        compute_metrics=compute_accuracy,
    )
  print("fff")  
  return trainer

In [ ]:
trainer = training(0)
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

In [ ]:
model.save_adapter("./final_adapter", "rotten_tomatoes")

!ls -lh final_adapter